---
title: "Agents - AIMA - Russell & Norvig (Part2)"
format: html
page-layout: full
code-line-numbers: true
code-block-border: true
toc: true
toc-location: left
number-sections: true
jupyter: python3
---

# Part2 - Case Study

In [1]:
import collections, copy, random
import numbers

In [2]:
class Thing:
    """This represents any physical object that can appear in an Environment.
    You subclass Thing to get the things you want. Each thing can have a
    .__name__  slot (used for output only)."""

    def __repr__(self):
        return '<{}>'.format(getattr(self, '__name__', self.__class__.__name__))

    def is_alive(self):
        """Things that are 'alive' should return true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Display the agent's internal state. Subclasses should override."""
        print("I don't know how to show_state.")

In [3]:
class Agent(Thing):
    """An Agent is a subclass of Thing with one required instance attribute 
    (aka slot), .program, which should hold a function that takes one argument,
    the percept, and returns an action. (What counts as a percept or action 
    will depend on the specific environment in which the agent exists.)
    Note that 'program' is a slot, not a method. If it were a method, then the
    program could 'cheat' and look at aspects of the agent. It's not supposed
    to do that: the program can only look at the percepts. An agent program
    that needs a model of the world (and of the agent itself) will have to
    build and maintain its own model. There is an optional slot, .performance,
    which is a number giving the performance measure of the agent in its
    environment."""

    def __init__(self, program=None):
        self.alive = True
        self.bump = False
        self.holding = []
        self.performance = 0
        if program is None or not isinstance(program, collections.abc.Callable):
            print("Can't find a valid program for {}, falling back to default.".format(self.__class__.__name__))

            def program(percept):
                return eval(input('Percept={}; action? '.format(percept)))

        self.program = program

    def can_grab(self, thing):
        """Return True if this agent can grab this thing.
        Override for appropriate subclasses of Agent and Thing."""
        return False

In [4]:
class Food(Thing):
    pass

class Water(Thing):
    pass

# The Agent

![Cool dog](https://gifgun.files.wordpress.com/2015/07/wpid-wp-1435860392895.gif)


In [5]:
class BlindDog(Agent):
    location = 1
    
    def movedown(self):
        self.location += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

Now its time to implement a <b>program</b> module for our dog. A program controls how the dog acts upon its environment. Our program will be very simple, and is shown in the table below.

| Percept | Action |
| :------- | :----------- |
| Feel Food | eat |
| Feel Water | drink |
| Feel Nothing | move down |

In [6]:
def program(percepts):
    '''Returns an action based on the dog's percepts'''
    for p in percepts:
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
            
    return 'move down'

# The Environment
 - A park is an example of an environment because the dog can perceive and act upon it.
 - We will start with a 1D park where the dog can only move down.

In [7]:
class Environment:
    """Abstract class representing an Environment. 'Real' Environment classes
    inherit from this. Your Environment will typically need to implement:
        percept:           Define the percept that an agent sees.
        execute_action:    Define the effects of executing an action.
                           Also update the agent.performance slot.
    The environment keeps a list of .things and .agents (which is a subset
    of .things). Each agent has a .performance slot, initialized to 0.
    Each thing has a .location slot, even though some environments may not
    need this."""

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Return the percept that the agent sees at this point. (Implement this.)"""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """Change the world to reflect this action. (Implement this.)"""
        raise NotImplementedError

    def default_location(self, thing):
        """Default location to place a new thing with unspecified location."""
        return None

    def exogenous_change(self):
        """If there is spontaneous change in the world, override this."""
        pass

    def is_done(self):
        """By default, we're done when we can't find a live agent."""
        return not any(agent.is_alive() for agent in self.agents)

    def step(self):
        """Run the environment for one time step. If the
        actions and exogenous changes are independent, this method will
        do. If there are interactions between them, you'll need to
        override this method."""
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)
                
            self.exogenous_change()

    def run(self, steps=1000):
        """Run the Environment for given number of time steps."""
        for step in range(steps):
            if self.is_done():
                print("All done!")
                return
            self.step()

    def list_things_at(self, location, tclass=Thing):
        """Return all things exactly at a given location."""
        if isinstance(location, numbers.Number):
            return [thing for thing in self.things
                    if thing.location == location and isinstance(thing, tclass)]
        return [thing for thing in self.things
                if all(x == y for x, y in zip(thing.location, location)) and isinstance(thing, tclass)]

    def some_things_at(self, location, tclass=Thing):
        """Return true if at least one of the things at location
        is an instance of class tclass (or a subclass)."""
        return self.list_things_at(location, tclass) != []

    def add_thing(self, thing, location=None):
        """Add a thing to the environment, setting its location. For
        convenience, if thing is an agent program we make a new agent
        for it. (Shouldn't need to override this.)"""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        if thing in self.things:
            print("Can't add the same thing twice")
        else:
            thing.location = location if location is not None else self.default_location(thing)
            self.things.append(thing)
            if isinstance(thing, Agent):
                thing.performance = 0
                self.agents.append(thing)

    def delete_thing(self, thing):
        """Remove a thing from the environment."""
        try:
            self.things.remove(thing)
        except ValueError as e:
            print(e)
            print("  in Environment delete_thing")
            print("  Thing to be removed: {} at {}".format(thing, thing.location))
            print("  from list: {}".format([(thing, thing.location) for thing in self.things]))
        if thing in self.agents:
            self.agents.remove(thing)

In [8]:
class Park(Environment):
    
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
            
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.

    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

# Simulation

In [9]:
park = Park()
dog = BlindDog(program)
dogfood = Food()
water = Water()

park.add_thing(dog, 1)
park.add_thing(dogfood, 5)
park.add_thing(water, 7)

In [10]:
park.run(5)

BlindDog decided to move down at location: 1
BlindDog decided to move down at location: 2
BlindDog decided to move down at location: 3
BlindDog decided to move down at location: 4
BlindDog ate Food at location: 5


In [11]:
park.run(5)

BlindDog decided to move down at location: 5
BlindDog decided to move down at location: 6
BlindDog drank Water at location: 7
All done!


In [12]:
park.add_thing(water, 15)
park.run(10)

BlindDog decided to move down at location: 7
BlindDog decided to move down at location: 8
BlindDog decided to move down at location: 9
BlindDog decided to move down at location: 10
BlindDog decided to move down at location: 11
BlindDog decided to move down at location: 12
BlindDog decided to move down at location: 13
BlindDog decided to move down at location: 14
BlindDog drank Water at location: 15
All done!


# Agents in 2D Environment

In [13]:
class Direction:
    """A direction class for agents that want to move in a 2D plane
        Usage:
            d = Direction("down")
            To change directions:
            d = d + "right" or d = d + Direction.R #Both do the same thing
            Note that the argument to __add__ must be a string and not a Direction object.
            Also, it (the argument) can only be right or left."""

    R = "right"
    L = "left"
    U = "up"
    D = "down"

    def __init__(self, direction):
        self.direction = direction

    def __repr__(self):
        return f"<<{self.direction}>>"

    def __add__(self, heading):
        """
        >>> d = Direction('right')
        >>> l1 = d.__add__(Direction.L)
        >>> l2 = d.__add__(Direction.R)
        >>> l1.direction
        'up'
        >>> l2.direction
        'down'
        >>> d = Direction('down')
        >>> l1 = d.__add__('right')
        >>> l2 = d.__add__('left')
        >>> l1.direction == Direction.L
        True
        >>> l2.direction == Direction.R
        True
        """
        if self.direction == self.R:
            return {
                self.R: Direction(self.D),
                self.L: Direction(self.U),
            }.get(heading)
        elif self.direction == self.L:
            return {
                self.R: Direction(self.U),
                self.L: Direction(self.D),
            }.get(heading)
        elif self.direction == self.U:
            return {
                self.R: Direction(self.R),
                self.L: Direction(self.L),
            }.get(heading)
        elif self.direction == self.D:
            return {
                self.R: Direction(self.L),
                self.L: Direction(self.R),
            }.get(heading)

    def move_forward(self, from_location):
        """
        >>> d = Direction('up')
        >>> l1 = d.move_forward((0, 0))
        >>> l1
        (0, -1)
        >>> d = Direction(Direction.R)
        >>> l1 = d.move_forward((0, 0))
        >>> l1
        (1, 0)
        """
        # get the iterable class to return
        iclass = from_location.__class__
        x, y = from_location
        if self.direction == self.R:
            return iclass((x + 1, y))
        elif self.direction == self.L:
            return iclass((x - 1, y))
        elif self.direction == self.U:
            return iclass((x, y - 1))
        elif self.direction == self.D:
            return iclass((x, y + 1))

In [14]:
d = Direction("right")
d2 = d + "right"
print(d2)
d2 = d2 + "right"
print(d2)
d2 = d2 + "left"
print(d2)

<<down>>
<<left>>
<<down>>


In [15]:
d = Direction("right")
d.move_forward((10,20))

(11, 20)

In [16]:
d = Direction("down")
d.move_forward((10,20))

(10, 21)

In [17]:
# 2D version but still moves only down

class BlindDog2(Agent):
    
    location = [0,1] # change location to a 2d value
    direction = Direction("down") # variable to store the direction our dog is facing
    
    def movedown(self):
        self.location[1] += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

In [18]:
park = Park()
dog = BlindDog2(program)
dogfood = Food()
water = Water()
park.add_thing(dog, [0,1])
park.add_thing(dogfood, [0,5])
park.add_thing(water, [0,7])
morewater = Water()
park.add_thing(morewater, [0,15])
print("BlindDog starts at (1,1) facing downwards, lets see if he can find any food!")

BlindDog starts at (1,1) facing downwards, lets see if he can find any food!


In [19]:
park.run(5)

BlindDog2 decided to move down at location: [0, 1]
BlindDog2 decided to move down at location: [0, 2]
BlindDog2 decided to move down at location: [0, 3]
BlindDog2 decided to move down at location: [0, 4]
BlindDog2 ate Food at location: [0, 5]


In [20]:
park.run(5)

BlindDog2 decided to move down at location: [0, 5]
BlindDog2 decided to move down at location: [0, 6]
BlindDog2 drank Water at location: [0, 7]
BlindDog2 decided to move down at location: [0, 7]
BlindDog2 decided to move down at location: [0, 8]


In [21]:
park.run(10)

BlindDog2 decided to move down at location: [0, 9]
BlindDog2 decided to move down at location: [0, 10]
BlindDog2 decided to move down at location: [0, 11]
BlindDog2 decided to move down at location: [0, 12]
BlindDog2 decided to move down at location: [0, 13]
BlindDog2 decided to move down at location: [0, 14]
BlindDog2 drank Water at location: [0, 15]
All done!


## PROGRAM - EnergeticBlindDog

Let's make our dog turn or move forwards at random. However, our dog is blind so the dog wouldn't know which way to turn - the dog just have to try arbitrarily.

<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Feel Food </td>
        <td>Feel Water</td>
        <td>Feel Nothing</td>
   </tr>
   <tr>
       <td><b>Action:</b> </td>
       <td>eat</td>
       <td>drink</td>
       <td>
       <table>
           <tr>
               <td><b>Remember being at Edge : </b></td>
               <td>At Edge</td>
               <td>Not at Edge</td>
           </tr>
           <tr>
               <td><b>Action : </b></td>
               <td>Turn Left / Turn Right <br> ( 50% - 50% chance )</td>
               <td>Turn Left / Turn Right / Move Forward <br> ( 25% - 25% - 50% chance )</td>
           </tr>
       </table>
       </td>
   </tr>
        
</table>

In [22]:
class EnergeticBlindDog(Agent):
    location = [0,1]
    direction = Direction("down")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False
        

In [23]:
def program(percepts):
    '''Returns an action based on it's percepts'''
    
    #print(f" Percepts: {percepts}")
    
    for p in percepts: # first eat or drink - you're a dog!
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
        if isinstance(p,Bump): # then check if you are at an edge and have to turn
            choice = random.choice((1,2));
        else:
            choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
        
    if choice == 1:
        return 'turnright'
    elif choice == 2:
        return 'turnleft'
    else:
        return 'moveforward'

In [24]:
class Bump(Thing):
    pass


In [25]:
class Park2D(Environment):

    def __init__(self, width=10, height=10):
        super().__init__()

        self.width = width
        self.height = height
        self.x_start, self.y_start = (0, 0)
        self.x_end, self.y_end = (self.width, self.height)
    
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall
        
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
            
        if not self.is_inbounds(loc):
            things.append(Bump())
            
        return things

    def is_inbounds(self, location):
        x, y = location
        return not (x < self.x_start or x > self.x_end - 1 or y < self.y_start or y > self.y_end - 1)
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        
        if action == 'turnright':
            print('  {} decided to {} at location: {} Current direction {}'.format(
                str(agent)[1:-1], action, agent.location, agent.direction))
            agent.turn(Direction.R)
            
        elif action == 'turnleft':
            print('  {} decided to {} at location: {} Current direction {}'.format(
                str(agent)[1:-1], action, agent.location, agent.direction))
            agent.turn(Direction.L)
            
        elif action == 'moveforward':
            print('  {} decided to move {}wards at location: {}  Current direction {}'.format(
                str(agent)[1:-1], agent.direction.direction, agent.location, agent.direction))
            agent.moveforward()     
            
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.

    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

In [26]:
random.seed(1)

park = Park2D(5,5)
dog = EnergeticBlindDog(program)
dogfood = Food()
water = Water()

park.add_thing(dog, [0,0])
park.add_thing(dogfood, [1,2])
park.add_thing(water, [0,1])

morewater = Water()
morefood = Food()

park.add_thing(morewater, [2,4])
park.add_thing(morefood, [4,3])

print("dog started at [0,0], facing down. Let's see if he found any food or water!")

dog started at [0,0], facing down. Let's see if he found any food or water!


In [28]:
park.run(10)

  EnergeticBlindDog decided to turnright at location: [4, 0] Current direction <<left>>
  EnergeticBlindDog decided to turnleft at location: [4, 0] Current direction <<up>>
  EnergeticBlindDog decided to move leftwards at location: [4, 0]  Current direction <<left>>
  EnergeticBlindDog decided to move leftwards at location: [3, 0]  Current direction <<left>>
  EnergeticBlindDog decided to move leftwards at location: [2, 0]  Current direction <<left>>
  EnergeticBlindDog decided to turnleft at location: [1, 0] Current direction <<left>>
  EnergeticBlindDog decided to move downwards at location: [1, 0]  Current direction <<down>>
  EnergeticBlindDog decided to turnright at location: [1, 1] Current direction <<down>>
  EnergeticBlindDog decided to move leftwards at location: [1, 1]  Current direction <<left>>
  EnergeticBlindDog decided to turnleft at location: [0, 1] Current direction <<left>>
